In [1]:
import datetime
import locale
locale.setlocale(locale.LC_ALL, '')

In [2]:
from context import openroute

# ors_ready = openroute.auto_switch()

# if ors_ready:
#     print('Using: {}'.format('azure' if openroute.api.is_online() else 'localhost'))
ors_ready = False

In [3]:
time_constraint = 30000

dataset = 'munchen'
ant_count = 971
iterations = 0
runtime = 0

path = [49, 48, 52, 56, 59, 57, 58, 64, 55, 47, 43, 51, 69, 75, 77, 81, 83, 87, 76, 78, 79, 72, 67, 63, 62, 54, 50, 44, 46, 40, 41, 49]

extra_html = f"""
    <tr>
        <td>Ants:</td><td align=right>{ant_count:n}</td>
    </tr>
"""
#     <tr>
#         <td>Iterations:</td><td align=right>{iterations:n}</td>
#     </tr>
# """

In [4]:
import json

with open(f"../data/{dataset}.json", 'r') as infile:
    data = json.load(infile)

geocaches = [[lat, lon] for lon, lat in data['metadata']['query']['locations']]
cost_matrix = data['durations']

In [5]:
import math

cost = 0

for index in range(len(path)-1):
    from_id = path[index]
    to_id = path[index+1]
    cost += cost_matrix[from_id][to_id]

cost = math.ceil(cost)

In [6]:
import folium
from algo_results import create_marker, create_circle_marker, fit_locations, style_function, create_legend

In [7]:
my_map = folium.Map(tiles=None, location=[52.498340, 13.358513], zoom_start=14, zoom_control=False)
folium.TileLayer(tiles='Stamen Toner', opacity=0.15).add_to(my_map)
fit_locations(my_map, geocaches)
pass

In [8]:
show_path = True
use_penultimate = path[0] == path[-1]
start_node = path[0]
last_node = path[-2] if use_penultimate else path[-1]
visited_nodes = len(path) - 1 if use_penultimate else len(path)

for index, coords in enumerate(geocaches):
    color = '#787c99'
    if show_path:
        if index == start_node: # start
            color = '#41a6b5'
        elif index == last_node: # last node before start
            color = '#9d7cd8'
        elif index in path: # other visited
            color = '#3c7da2'
        create_marker(path, coords, index, color, radius=10).add_to(my_map)
    else:
        create_marker([], coords, index, color, radius=10).add_to(my_map)

In [9]:
from tqdm import tqdm

optimal_locations_latlon = [[geocaches[id][0], geocaches[id][1]] for id in path]
optimal_locations = [[geocaches[id][1], geocaches[id][0]] for id in path]
optimal_routes = []

In [10]:
line_color = '#bb616b'

if ors_ready:
    for index in tqdm(range(len(optimal_locations)-1)):
        request = {
            'coordinates': optimal_locations[index:index+2],
            'geometry': True,
            'instructions': False
        }
        optimal_route = openroute.request_directions(request, format_out='/geojson')

        if optimal_route and not 'error' in optimal_route:
            folium.features.GeoJson(data=optimal_route,
                                    name='Optimal route',
                                    style_function=style_function(line_color),
                                    overlay=True).add_to(my_map)
            optimal_routes.append(optimal_route)
        else:
            folium.PolyLine(optimal_locations_latlon[index:index+2], color=line_color, weight=3, opacity=1).add_to(my_map)
else:
    folium.PolyLine(optimal_locations_latlon, color=line_color, weight=3, opacity=1).add_to(my_map)

In [11]:
cost_str = str(datetime.timedelta(seconds=round(cost)))

time_constraint_html = ''
if time_constraint:
    time_constraint_html = f"""
        <tr>
            <td>Time constraint:</td>
            <td align=right>{str(datetime.timedelta(seconds=time_constraint))}</td>
        </tr>
    """

score = visited_nodes + (1 - cost / time_constraint)

legend = create_legend(f"""
    <table width=100%>
        {extra_html}
        <tr>
            <td>Runtime:</td><td align=right>{runtime:n}s</td>
        </tr>
        {time_constraint_html}
        <tr>
            <td>Duration:</td>
            <td align=right>{cost_str}</td>
        </tr>
        <tr>
            <td>Cost:</td>
            <td align=right>{cost}</td>
        </tr>
        <tr>
            <td>Visited nodes:</td>
            <td align=right>{visited_nodes}</td>
        </tr>
        <tr>
            <td>Score:</td>
            <td align=right>{score:.4f}</td>
        </tr>
    </table>
    <div class="marker-description">
        <span class="start">Start node</span>
        <span class="last">{"Penultimate" if use_penultimate else "Last"} node</span>
    </div>
""")

my_map.get_root().html.add_child(legend)

pass

In [12]:
my_map